In [1]:
import torch
import torch.nn as nn


import sys

sys.path.append('app/src')
sys.path.append('./src')
from util import sparse_tensor_stats


from sparse_evaluation_4 import SparseEvaluation  
from zono_sparse_gen import ZonoSparseGeneration

from abstract_relu import AbstractReLU
from sparse_addition_2 import SparseAddition

/home/guillaume/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-08 17:13:14,521	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [21]:
input = torch.randn(1,3)
input = torch.where(input>0,input,0)
print(input)
_, new_sparse = ZonoSparseGeneration(input,from_trash=True,start_index=5).total_zono()

tensor([[2.4162, 0.0000, 1.2699]])


In [22]:
new_sparse

tensor(indices=tensor([[5, 6],
                       [0, 2]]),
       values=tensor([2.4162, 1.2699]),
       size=(7, 3), nnz=2, layout=torch.sparse_coo)

In [23]:
new_sparse.to_dense()

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [2.4162, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.2699]])

In [8]:
import ray

# Simuler ray.remote pour cet exemple
def remote_decorator(num_gpus=None, num_cpus=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            resources = []
            if num_gpus is not None:
                resources.append(f"{num_gpus} GPUs")
            if num_cpus is not None:
                resources.append(f"{num_cpus} CPUs")
            print(f"Running with: {', '.join(resources)}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

class DecoratedClass:
    def __init__(self, cls, gpu_decorator, cpu_decorator):
        self.decorated_class = self._create_decorated_class(cls, gpu_decorator, cpu_decorator)

    def _create_decorated_class(self, cls, gpu_decorator, cpu_decorator):
        class Decorated(cls):
            pass
        
        for attr_name in dir(cls):
            attr = getattr(cls, attr_name)
            if callable(attr) and not attr_name.startswith("__"):
                decorated_method = gpu_decorator(attr)
                decorated_method = cpu_decorator(decorated_method)
                setattr(Decorated, attr_name, decorated_method)
                
        return Decorated

    def __call__(self, *args, **kwargs):
        return self.decorated_class(*args, **kwargs)

# Exemple de décorateurs
gpu_decorator = remote_decorator(num_gpus=1)
cpu_decorator = remote_decorator(num_cpus=4)

# Utilisation
DecoratedMyClass = DecoratedClass(MyClass, gpu_decorator, cpu_decorator)

# Instanciation et appel de la méthode décorée
instance = DecoratedMyClass()
print(instance.my_method())
print(instance)


Running with: 4 CPUs
Running with: 1 GPUs
Hello from MyClass
<__main__.DecoratedClass._create_decorated_class.<locals>.Decorated object at 0x72155eee4370>


In [13]:
ray.shutdown()

In [12]:
import ray

# Supposons que votre classe SparseWorker ressemble à ceci :
class SparseWorker:
    def __init__(self, param1, param2):
        self.param1 = param1
        self.param2 = param2

    def work(self):
        return self.param1 + self.param2

# Fonction pour instancier la classe avec des paramètres dynamiques
def create_sparse_worker(num_cpus,num_gpus, *args, **kwargs):
    SparseWorkerRemote = ray.remote(num_cpus=num_cpus,num_gpus=num_gpus)(SparseWorker)
    return SparseWorkerRemote.remote(*args, **kwargs)

# Initialiser Ray
ray.init()

# Créez une instance de SparseWorker avec un nombre personnalisé de CPUs
worker = create_sparse_worker(2,1, "param1_value", "param2_value")

# Appelez une méthode de la classe
result = ray.get(worker.work.remote())
print(result)


2024-07-10 15:12:17,051	INFO worker.py:1753 -- Started a local Ray instance.


(autoscaler +34m2s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +34m2s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


KeyboardInterrupt: 

In [5]:
import sys
import argparse
import torch
import torch.nn as nn
from torch.sparse import FloatTensor
from typing import List, Union, Tuple, Callable

# Add paths for imports
sys.path.append('app/src')
sys.path.append('./src')


def list_of_shape(tensor: torch.Tensor) -> List[int]:
    """Returns the shape of a tensor as a list."""
    tensor.clone().detach()
    return list(tensor.shape)

class ZonoSparseGeneration:
    """This class generates a sparse representation of an abstract domain."""
    
    def __init__(self, input: FloatTensor, noise_intensity : Union[float, torch.Tensor]=0, noise_type: str = 'additive', 
                 indices=None, from_trash=False, start_index=None):
        self.input = input.to('cpu')
        self.noise_intensity = torch.tensor(noise_intensity).to('cpu')
        self.noise_type = noise_type
        self.input_shape = list_of_shape(input)
        self.indices = indices
        self.from_trash = from_trash
        self.start_index = start_index

    def total_zono(self):
        """Generates a sparse zonotope."""
        if not self.from_trash:
            dim_input = torch.tensor(self.input_shape).numel()

            if dim_input == 1:
                global_storage = {'indices': [], 'values': []}

                if self.indices is None:
                    num_elements = self.input_shape[0]
                    self.indices = torch.arange(1, num_elements, 1)
                else:
                    self.indices = self.indices.to('cpu')
                    #self.indices = torch.tensor(self.indices)
                    num_elements = self.indices.numel()

                if len(self.noise_intensity.flatten()) == 1:
                    self.noise_intensity = self.noise_intensity * torch.ones_like(self.indices)
                else:
                    print(self.noise_intensity.size())
                    print(self.indices.size())
                    assert self.noise_intensity.size() == self.indices.size(), 'the length of noise intensity must be one or equal to indices shape'

                for i in range(num_elements):
                    global_storage['indices'].append([self.indices[i], self.indices[i]])
                    global_storage['values'].append(self.noise_intensity[i])

                indice_tensor = torch.tensor(global_storage['indices'], dtype=torch.int32).t()
                values_tensor = torch.tensor(global_storage['values'], dtype=torch.float32)
                sparse_zonotope = torch.sparse_coo_tensor(indice_tensor, values_tensor, size=(self.input_shape[0], self.input_shape[0])).coalesce()

                return self.input, sparse_zonotope.to_dense()

            if dim_input == 2:
                self.input = self.input.unsqueeze(0)
                self.input_shape = list_of_shape(self.input)
            
            if dim_input == 4:
                self.input = self.input.squeeze(0)
                print("WARNING: Trying to generate abstract Sparse tensor from a batch, only the first element will be used")
                self.input_shape = list_of_shape(self.input)

            if self.indices is None:
                assert len(self.noise_intensity.flatten()) == 1, 'Shape of noise and indices do not match'
                num_elements = self.input_shape[0]
                self.indices = torch.arange(1, num_elements, 1)
                global_storage = {'indices': [], 'values': []}
                num_elements = self.input_shape[0] * self.input_shape[1] * self.input_shape[2]

                for i in range(num_elements):
                    dim_3 = i // (self.input_shape[1] * self.input_shape[2])
                    rem = i % (self.input_shape[1] * self.input_shape[2])
                    dim_1 = rem // self.input_shape[1]
                    dim_2 = rem % self.input_shape[2]
                    global_storage['indices'].append([i, dim_3, dim_1, dim_2])
                    global_storage['values'].append(self.noise_intensity)

                indice_tensor = torch.tensor(global_storage['indices'], dtype=torch.int32).t()
                values_tensor = torch.tensor(global_storage['values'], dtype=torch.float32)
                sparse_zonotope = torch.sparse_coo_tensor(indice_tensor, values_tensor, size=(num_elements, self.input_shape[0], self.input_shape[1], self.input_shape[2])).coalesce()

            else:
                self.indices = torch.tensor(self.indices).to('cpu')
                #assert len(self.indices) == len(self.noise_intensity), 'Length of Noise_intensity and indices mismatch'
                global_storage = {'indices': [], 'values': []}
                num_elements = len(self.indices)

                for i in range(num_elements):
                    if len(self.indices[i]) == 2:
                        global_storage['indices'].append(torch.cat((torch.tensor([i, 0]), self.indices[i])).tolist())
                    else:
                        global_storage['indices'].append(torch.cat((torch.tensor([i]), self.indices[i])).tolist())
                    global_storage['values'].append(self.noise_intensity[i])

                indice_tensor = torch.tensor(global_storage['indices'], dtype=torch.int32).t()
                values_tensor = torch.tensor(global_storage['values'], dtype=torch.float32)
                print(indice_tensor)
                print(values_tensor)

                sparse_zonotope = torch.sparse_coo_tensor(indice_tensor, values_tensor, size=(num_elements, self.input_shape[0], self.input_shape[1], self.input_shape[2])).coalesce()

            return self.input, sparse_zonotope

        if self.from_trash:
            if not self.start_index:
                print('Warning, start_index is 0, should start at the depth of abstract domain')
                self.start_index = 0

            global_storage = {'indices': [], 'values': []}
            indices = torch.nonzero(self.input)
            if len(indices)==0: 
                return self.input, None
          

            for i, indice in enumerate(indices):
                sparse_indice = torch.cat((torch.tensor([i + self.start_index]), indice[1:])).tolist()
                global_storage['indices'].append(sparse_indice)
                global_storage['values'].append(self.input[tuple(indice.tolist())])
            
            indice_tensor = torch.tensor(global_storage['indices'], dtype=torch.int32).t()
            values_tensor = torch.tensor(global_storage['values'], dtype=torch.float32)
            dim = tuple(torch.cat((torch.tensor([len(indices)+self.start_index]), torch.tensor(list_of_shape(self.input.squeeze(0))))))

            sparse_zonotope = torch.sparse_coo_tensor(indice_tensor, values_tensor, size=dim).coalesce()

            return self.input, sparse_zonotope

In [8]:
test  = torch.randn(1,3,24,24)
_,zonotop = ZonoSparseGeneration(input=test, noise_intensity=test.to_sparse().values(),indices = test.to_sparse().indices()).total_zono()

tensor([[ 0,  1,  2,  3],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  1],
        ...,
        [ 0,  2, 23, 21],
        [ 0,  2, 23, 22],
        [ 0,  2, 23, 23]], dtype=torch.int32)
tensor([ 0.8147,  0.4032, -1.0225, -0.2999])


/tmp/ipykernel_2764178/3528852022.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.noise_intensity = torch.tensor(noise_intensity).to('cpu')
/tmp/ipykernel_2764178/3528852022.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.indices = torch.tensor(self.indices).to('cpu')


RuntimeError: number of dimensions must be sparse_dim (1729) + dense_dim (0), but got 4

In [63]:
class ZonoSparseGeneration:
    def __init__(self):
        self.global_storage = {}
        self.global_storage['indices'] =[]
        self.global_storage['values'] = []
        pass
    def zono_from_tensor(self,noise_intensity):
        assert noise_intensity.size(0)==1,'First dimension size must be 1'
        noise_intensity = noise_intensity.to_sparse()
        noise_intensity.indices()[0] = torch.arange(noise_intensity._nnz())
        size =(noise_intensity._nnz(),*noise_intensity.size()[1:])

        noise_intensity = torch.sparse_coo_tensor(noise_intensity.indices(),noise_intensity.values(),size = size)
        noise_intensity.coalesce()

        return(noise_intensity)

    def zono_from_noise_level_and_tensor(self,noise_level,tensor):
        
        noise_intensity = noise_level*torch.ones_like(tensor)
        zonotope = self.zono_from_tensor(noise_intensity=noise_intensity)
        return zonotope



   

         

            


In [67]:
test = ZonoSparseGeneration().zono_from_tensor( noise_intensity=torch.randn(1,3
                                                                           ))
print(test.to_dense())

(3, 3)
tensor([[-0.3511,  0.0000,  0.0000],
        [ 0.0000, -0.5359,  0.0000],
        [ 0.0000,  0.0000, -0.6982]])


In [66]:
test_2 = ZonoSparseGeneration().zono_from_noise_level_and_tensor(noise_level=0.01,tensor=torch.randn(1,2))
print(test_2.to_dense())

(2, 2)
tensor([[0.0100, 0.0000],
        [0.0000, 0.0100]])
